# **---  `DatasetEphy`: container of neurophysiological data ---**
---
In this tutorial we're mainly going to see how to define a container hosting the neurophysiological data of one or several subjects.

In [ ]:
import os

import numpy as np
import xarray as xr
import pandas as pd

from mne import EpochsArray, create_info

from frites.dataset import DatasetEphy

import matplotlib.pyplot as plt

---
# **--- Structure of the `DatasetEphy` ---**

## Signature of the `DatasetEphy`

**`DatasetEphy(x, y='...', times='...', roi='...')`** where :
---

1. **`x` = The brain data**
    * **[Description] :** list containing the brain data of one or multiple subjects / sessions
    * **[Sizes] :** each element of the list is the epoched brain data. For example, for two subjects it would be : $[(n_{epochs}, n_{channels}, n_{times})_{subject_1}, (n_{epochs}, n_{channels}, n_{times})_{subject_2}]$
2. **`y` = The external variable**
    * **[Description] :** list containing the external variable (e.g. stimulus type, behavioral model, reaction time etc.) of one or multiple subjects / sessions
    * **[Goal] :** the goal is then to ask if the variables `x` (brain data) and the external variable `y` shared information. Said differently, if there's differences in the brain activity according to the stimulus (\~decoding) or if the brain data correlates with the behavioral model or reaction time (\~regression)
    * **[Sizes] :** each element of the list is the external variable of a single subject. For example, the reaction time for two subjects : $[RT_{subject_1}, RT_{subject_2}] = [(n_{epochs},)_{subject_1}, (n_{epochs},)_{subject_2}]$
3. **`times` = The time vector**
    * **[Description] :** a single time vector
    * **[Alternatives] :** in fact it can be anything, like frequencies (e.g. PSD)
4. **`roi` = spatial dimension**
    * **[Description] :** list containing a spatial description of each subject (e.g. the name of the channels for i/M/EEG, the name of brain regions etc.)
    * **[Sizes] :** each element of the list describes the channels / ROI of a single subject. For example, still for two subjects : $[ROI_{subject_1}, ROI_{subject_2}] = [(n_{channels},)_{subject_1}, (n_{channels},)_{subject_2}]$
    
## Good to know

* It works for one / many subjects
* It works for one / many sessions of a single subject (or animals like monkeys)
* All subjects can have a different number of trials (or epochs) such as a different number of channels. The channels can also be located in different brain regions
* However, the time vector **should be the same across all subjects or sessions**

---
# **0 - Functions**

In [ ]:
###############################################################################
###############################################################################
#                 Load the data of a single subject
###############################################################################
###############################################################################

def load_ss(root, subject_nb):
    """Load the data of a single subject.
    
    Parameters
    ----------
    root : string
        Root path where the dataset is stored
    subject_nb : int
        Subject number [0, 12]
    
    Returns
    -------
    hga : xarray.DataArray
        Xarray containing the high-gamma activity
    anat : pandas.DataFrame
        Table containing the anatomical informations
    beh : pandas.DataFrame
        Table containing the behavioral informations
    """
    print(f"Loading the data of subject #{subject_nb}")

    # load the high-gamma activity
    file_hga = os.path.join(root, 'hga', f'hga_s-{subject_nb}.nc')
    hga = xr.load_dataarray(file_hga)

    # load the name of the brain regions
    file_anat = os.path.join(root, 'anat', f'anat_s-{subject_nb}.xlsx')
    anat = pd.read_excel(file_anat)

    # load the behavior
    file_beh = os.path.join(root, 'beh', f'beh_s-{subject_nb}.xlsx')
    beh = pd.read_excel(file_beh)
    
    return hga, anat, beh


---
# **1 - Define a `DatasetEphy` using Xarray**
## 1.1 Load the data of a single subject

In [ ]:
###############################################################################
root = '/home/etienne/DATA/Study/PBLT/gdr/'
###############################################################################

# load the data of subject #2
subject_nb = 2
hga, anat, beh = load_ss(root, subject_nb)

## 1.2 Define the `DatasetEphy`

In [ ]:
ds = DatasetEphy([hga.copy()])

# checkout the warnings !!

## 1.3 Define the `DatasetEphy` with dimensions

In [ ]:
"""
When defining the `DatasetEphy`, we can specify the name of the dimensions
for the time (`times=`) and for the space (`roi`)
"""
ds = DatasetEphy([hga.copy()], times='times', roi='channels')
ds

# - Checkout the html output !
# - "Supported MI definition" is none? wtf

## 1.4 Specify the stimulus type

In [ ]:
"""
Each trial contain the information of the stimulus type !

* -2 = "-1€"
* -1 = "-0€"
* +1 = "+0€"
* +2 = "+1€"
"""
hga['trials']

"""We can specify in the `DatasetEphy` the name of the dimension containing the
stimulus type
"""
ds = DatasetEphy([hga.copy()], y='trials', times='times', roi='channels')
ds

## 1.5 Define a `DatasetEphy` for all of the subjects

In [ ]:
# start by loading the data coming from multiple subjects
hga = []
for n_s in range(12):  # 12 subjects in total
    # load the data of a single subject
    _hga, _, _ = load_ss(root, n_s)
    
    # append the data to the list
    hga.append(_hga)

# create the dataset ephy
ds = DatasetEphy(hga, y='trials', times='times', roi='channels')
ds

# **2 - Define a `DatasetEphy` with NumPy**
## 2.1 Single subject

In [ ]:
# load the data of subject #2
subject_nb = 2
hga, anat, beh = load_ss(root, subject_nb)

# get the contact names
channels = hga['channels'].data

# get the time vector
times = hga['times'].data

# get the stimulus types
stim = hga['trials'].data

# get the hga as a NumPy array
hga_np = hga.data

# create the dataset ephy
ds = DatasetEphy(
    [hga_np], y=[stim], times=times, roi=[channels])
ds

## 2.2 Multi-subjects

In [ ]:
# start by loading the data coming from multiple subjects
hga, channels, stim = [], [], []
for n_s in range(12):  # 12 subjects in total
    # load the data of a single subject
    _hga, _, _ = load_ss(root, n_s)
    
    # get the contact names
    _channels = _hga['channels'].data

    # get the time vector
    times = _hga['times'].data

    # get the stimulus types
    _stim = _hga['trials'].data

    # get the hga as a NumPy array
    hga_np = _hga.data

    # append the data to the list
    hga.append(_hga)
    channels.append(_channels)
    stim.append(_stim)

# create the dataset ephy
ds = DatasetEphy(hga, y=stim, times=times, roi=channels)
ds

# **3 - Define a `DatasetEphy` using MNE-Python objects**

In [ ]:
# _____________________________ CONVERSION TO MNE _____________________________
# load the data of subject #2
subject_nb = 2
hga, anat, beh = load_ss(root, subject_nb)

# create the information used by MNE
info = create_info(
    hga['channels'].data.tolist(), hga.attrs['sfreq'], ch_types='seeg'
)

# create the epoch
epoch = EpochsArray(hga.data, info, tmin=hga['times'].data[0])

# ________________________________ DATASET EPHY _______________________________
ds = DatasetEphy([epoch], y=[hga['trials'].data])
ds

---
# **---- Test yourself ! ----**
## **1. Load fresh data !**

<div class="alert alert-warning"><p>

**[Instructions]** Load the data, behavior and anatomy of subject #0
</p></div>

In [ ]:
# write your answer

## **2. `DatasetEphy` with the Prediction Error**
### 2.1 Plot the Prediction error

In the behavioral table (output `beh` of the function `load_ss()`) there's a column called `'PE'` which stands for `prediction error`. This model is estimated using the behavior of the subject. It represents the mismatch between a prior expectation of the future outcome and the actual outcome obtained (i.e. $Outcome_{expected} - Outcome_{obtained}$).
    
Said differently, it's kind of a learning curve estimated trial after trial, where at the beginning the subject is not really good at predicting the outcome he's going to obtained (PE very high) but at the end, he nails it like a pro (PE at 0) ! 

The table `beh` also contains a column `block` indicating the session (or block).

<div class="alert alert-warning"><p>

**[Instructions]**
    
Let's start "easy" by plotting the PE, only for the first block. You should see it decreasing to almost 0 !
</p></div>

In [ ]:
# write your answer

### 2.2 Set the Prediction error as the trial coordinate

One typical question you can ask using the PE is **"what are the brain regions for which the brain data are modulated accordingly to the PE?"** or said differently, what are the brain regions involved during learning.

The first step to answer this question is to tell to the `DatasetEphy` that the external variable `y` is going to be the PE.

<div class="alert alert-warning"><p>

**[Instructions]**
    
1. Rename the dimension `trials` in the hga `DataArray` by `pe`
2. Fill this dimension `pe` with the values of the PE
</p></div>

In [ ]:
# write your answer

### 2.3 Define a `DatasetEphy` for a single subject

<div class="alert alert-warning"><p>

**[Instructions]**
    
Define a `DatasetEphy` for the subject #0 you just loaded and specify the coordinates of the PE (`y`), the time (`times`) and the spatial dimension (`roi`)
</p></div>

In [ ]:
# write your answer

In [ ]:
a = input('What is the `Supported MI definition`?')

if a == 'I(x; y (continuous)) (cc)':
    print("Yes, you're right !")
else:
    print('Nope ! Try again :)')

## **3. `DatasetEphy` with brain regions**
### 3.1 Drop channels - replace with brain region names

Here, we're going to replace the name of the channels by their corresponding brain region names and define a `DatasetEphy` with those names of brain regions.

<div class="alert alert-warning"><p>

**[Instructions]**
    
1. Get the name of the brain regions (`anat` output, column `roi`)
2. Rename the dimension name `channels` of the `hga` to be `parcels`
3. Fill this dimension `roi` with the name of the brain regions
4. Define the `DatasetEphy` and specify the the `roi` dimension is now called `parcels`
</p></div>

In [ ]:
# write your answer

### 3.2 Do the same, for all of the subjects

<div class="alert alert-warning"><p>

**[Instructions]**
    
Build a `DatasetEphy` with the 12 subjects, each having the PE dimension and the brain region names instead of the channel names
</p></div>

In [ ]:
# write your answer